In [1]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [2]:
# Define inputs
distance = ctrl.Antecedent(np.arange(0, 101, 1), 'distance')  # Distance to vehicle ahead
relative_speed = ctrl.Antecedent(np.arange(-50, 51, 1), 'relative_speed')  # Negative = vehicle ahead is slower

In [3]:
# Define outputs
throttle = ctrl.Consequent(np.arange(0, 11, 1), 'throttle')  # 0 = decrease, 10 = increase
brake = ctrl.Consequent(np.arange(0, 101, 1), 'brake')  # Brake pressure

In [4]:
# Membership functions for distance
distance['close'] = fuzz.trimf(distance.universe, [0, 0, 40])
distance['medium'] = fuzz.trimf(distance.universe, [30, 50, 70])
distance['far'] = fuzz.trimf(distance.universe, [60, 100, 100])

# Membership functions for relative speed
relative_speed['slower'] = fuzz.trimf(relative_speed.universe, [-50, -50, 0])
relative_speed['same'] = fuzz.trimf(relative_speed.universe, [-10, 0, 10])
relative_speed['faster'] = fuzz.trimf(relative_speed.universe, [0, 50, 50])

# Membership functions for throttle
throttle['decrease'] = fuzz.trimf(throttle.universe, [0, 0, 3])
throttle['maintain'] = fuzz.trimf(throttle.universe, [3, 5, 7])
throttle['increase'] = fuzz.trimf(throttle.universe, [7, 10, 10])

# Membership functions for brake
brake['low'] = fuzz.trimf(brake.universe, [0, 0, 30])
brake['medium'] = fuzz.trimf(brake.universe, [20, 50, 80])
brake['high'] = fuzz.trimf(brake.universe, [70, 100, 100])

In [5]:
# Define fuzzy rules
rule1 = ctrl.Rule(distance['close'] & relative_speed['slower'],
                  consequent=[throttle['decrease'], brake['medium']])

rule2 = ctrl.Rule(distance['close'] & relative_speed['same'],
                  consequent=[throttle['decrease'], brake['high']])

rule3 = ctrl.Rule(distance['medium'] & relative_speed['slower'],
                  consequent=[throttle['maintain'], brake['low']])

rule4 = ctrl.Rule(distance['medium'] & relative_speed['same'],
                  consequent=[throttle['maintain'], brake['low']])

rule5 = ctrl.Rule(distance['far'] & relative_speed['same'],
                  consequent=[throttle['increase'], brake['low']])

rule6 = ctrl.Rule(distance['far'] & relative_speed['faster'],
                  consequent=[throttle['increase'], brake['low']])

rule7 = ctrl.Rule(distance['far'] & relative_speed['slower'],
                  consequent=[throttle['maintain'], brake['low']])

In [6]:
# Create control system
acc_system = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])
acc_sim = ctrl.ControlSystemSimulation(acc_system)

In [7]:
# Example inputs
acc_sim.input['distance'] = 25       # Close distance
acc_sim.input['relative_speed'] = -20  # Vehicle ahead is slower

# Compute the output
acc_sim.compute()

# Show results
print(f"Throttle Adjustment: {acc_sim.output['throttle']:.2f}")
print(f"Brake Pressure: {acc_sim.output['brake']:.2f}")

Throttle Adjustment: 1.24
Brake Pressure: 50.00
